In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
import torch


print(torch.__version__)
print("GPU Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = "cpu"

2.0.1
GPU Available: False


In [3]:
import torch.nn as nn
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
batch_size = 64

torch.manual_seed(1)
np.random.seed(1)

In [6]:
import pickle

# To load the lists from the file:
with open(r'C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\EEG-To-Text-GAN\Data\EEG_Text_Pairs.pkl', 'rb') as file:
    EEG_word_level_embeddings = pickle.load(file)
    EEG_word_level_labels = pickle.load(file)

In [30]:
float_tensor = torch.tensor(EEG_word_level_embeddings, dtype=torch.float)

C:\Users\gxb18167\AppData\Local\Temp\ipykernel_24712\839738786.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\b\abs_abjetg6_iu\croot\pytorch_1686932924616\work\torch\csrc\utils\tensor_new.cpp:248.)
  float_tensor = torch.tensor(EEG_word_level_embeddings, dtype=torch.float)


In [49]:
float_tensor[0][0].shape

torch.Size([105, 8])

In [31]:
import torch
train_data = []
for i in range(len(float_tensor)):
   train_data.append([float_tensor[i], EEG_word_level_labels[i]])



trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=64)

In [166]:
#sanity check:
i1, l1 = next(iter(trainloader))
print(i1.shape, l1)

tensor([[[2.7698, 1.3693, 1.0006, 1.0377, 0.9263, 0.3586, 1.3011, 0.9097],
         [1.0833, 1.1161, 0.5786, 1.0576, 1.1416, 1.1544, 0.9489, 0.6407],
         [0.7564, 1.1634, 0.8472, 1.7551, 0.6858, 0.9577, 0.7568, 0.5076],
         [0.4106, 1.1224, 1.7265, 0.8217, 0.8020, 0.8461, 3.0502, 0.9161],
         [1.8313, 1.2387, 1.2132, 1.0831, 0.5694, 2.3264, 1.7901, 1.7870],
         [3.2848, 3.0052, 2.5046, 1.6938, 1.1088, 0.4108, 3.6942, 3.6750],
         [3.3399, 2.9089, 2.1167, 2.4074, 3.8752, 4.0677, 4.0108, 3.5303],
         [4.3200, 3.7503, 3.9125, 3.3102, 3.6240, 3.6315, 3.5991, 2.3649],
         [1.7955, 0.7117, 1.1090, 3.6161, 4.7058, 2.7662, 2.2068, 1.6651],
         [1.0571, 3.3795, 3.1622, 2.6551, 1.8112, 1.4100, 3.1329, 2.6095],
         [2.3858, 1.6953, 2.7671, 3.1854, 1.7641, 1.1121, 1.0452, 1.3722],
         [0.7494, 2.2713, 0.9478, 0.7662, 1.3037, 1.0658, 1.5393, 2.0574],
         [1.2024, 1.0218, 1.5682, 1.1326, 1.6254, 2.1454, 2.3601, 1.6923],
         [0.0000, 1.4240,

In [152]:
z_size = 100
image_size = (105, 8)
n_filters = 32

In [153]:
## Loss function and optimizers:
loss_fn = nn.BCELoss()

In [154]:
def create_noise(batch_size, z_size, mode_z):
    if mode_z == 'uniform':
        input_z = torch.rand(batch_size, z_size, 1, 1)*2 - 1
    elif mode_z == 'normal':
        input_z = torch.randn(batch_size, z_size, 1, 1)
    return input_z

mode_z = 'uniform'
fixed_z = create_noise(batch_size, z_size, mode_z).to(device)

def create_samples(g_model, input_z):
    g_output = g_model(input_z)
    images = torch.reshape(g_output, (batch_size, *image_size))
    return (images+1)/2.0

In [155]:
noise = create_noise(64, 100, "uniform")

In [156]:
noise.shape

torch.Size([64, 100, 1, 1])

In [157]:
def make_generator_network_wgan(input_size, n_filters):
    model = nn.Sequential(
        nn.ConvTranspose2d(input_size, n_filters*4, 4, 1, 0,
                           bias=False),
        nn.InstanceNorm2d(n_filters*4),
        nn.LeakyReLU(0.2),

        nn.ConvTranspose2d(n_filters*4, n_filters*2, 3, 2, 1, bias=False),
        nn.InstanceNorm2d(n_filters*2),
        nn.LeakyReLU(0.2),

        nn.ConvTranspose2d(n_filters*2, n_filters, 4, 2, 1, bias=False),
        nn.InstanceNorm2d(n_filters),
        nn.LeakyReLU(0.2),

        nn.ConvTranspose2d(n_filters, 1, 4, 2, 1, bias=False),
        nn.Tanh())
    return model

class DiscriminatorWGAN(nn.Module):
    def __init__(self, n_filters):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(1, n_filters, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2),

            nn.Conv2d(n_filters, n_filters*2, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(n_filters * 2),
            nn.LeakyReLU(0.2),

            nn.Conv2d(n_filters*2, n_filters*4, 3, 2, 1, bias=False),
            nn.InstanceNorm2d(n_filters*4),
            nn.LeakyReLU(0.2),

            nn.Conv2d(n_filters*4, 1, 4, 1, 0, bias=False),
            nn.Sigmoid())

    def forward(self, input):
        output = self.network(input)
        return output.view(-1, 1).squeeze(0)

In [158]:
gen_model = make_generator_network_wgan(z_size, n_filters).to(device)
disc_model = DiscriminatorWGAN(n_filters).to(device)

g_optimizer = torch.optim.Adam(gen_model.parameters(), 0.0002)
d_optimizer = torch.optim.Adam(disc_model.parameters(), 0.0002)

In [159]:
from torch.autograd import grad as torch_grad


def gradient_penalty(real_data, generated_data):
    batch_size = real_data.size(0)

    # Calculate interpolation
    alpha = torch.rand(real_data.shape[0], 1, 1, 1, requires_grad=True, device=device)
    interpolated = alpha * real_data + (1 - alpha) * generated_data

    # Calculate probability of interpolated examples
    proba_interpolated = disc_model(interpolated)

    # Calculate gradients of probabilities with respect to examples
    gradients = torch_grad(outputs=proba_interpolated, inputs=interpolated,
                           grad_outputs=torch.ones(proba_interpolated.size(), device=device),
                           create_graph=True, retain_graph=True)[0]

    gradients = gradients.view(batch_size, -1)
    gradients_norm = gradients.norm(2, dim=1)
    return lambda_gp * ((gradients_norm - 1)**2).mean()


In [160]:
## Train the discriminator
def d_train_wgan(x):
    disc_model.zero_grad()

    batch_size = x.size(0)
    x = x.to(device)

    # Calculate probabilities on real and generated data
    d_real = disc_model(x)
    input_z = create_noise(batch_size, z_size, mode_z).to(device)
    g_output = gen_model(input_z)
    d_generated = disc_model(g_output)
    d_loss = d_generated.mean() - d_real.mean() + gradient_penalty(x.data, g_output.data)
    d_loss.backward()
    d_optimizer.step()

    return d_loss.data.item()

In [161]:
## Train the generator
def g_train_wgan(x):
    gen_model.zero_grad()

    batch_size = x.size(0)
    input_z = create_noise(batch_size, z_size, mode_z).to(device)
    g_output = gen_model(input_z)

    d_generated = disc_model(g_output)
    g_loss = -d_generated.mean()

    # gradient backprop & optimize ONLY G's parameters
    g_loss.backward()
    g_optimizer.step()

    return g_loss.data.item()

In [162]:
epoch_samples_wgan = []
lambda_gp = 10.0
num_epochs = 100
torch.manual_seed(1)
critic_iterations = 5

for epoch in range(1, num_epochs+1):
    gen_model.train()
    d_losses, g_losses = [], []
    for i, (x, _) in enumerate(trainloader):
        for _ in range(critic_iterations):
            d_loss = d_train_wgan(x)
        d_losses.append(d_loss)
        g_losses.append(g_train_wgan(x))

    print(f'Epoch {epoch:03d} | D Loss >>'
          f' {torch.FloatTensor(d_losses).mean():.4f}')
    gen_model.eval()
    epoch_samples_wgan.append(
        create_samples(gen_model, fixed_z).detach().cpu().numpy())

RuntimeError: Calculated padded input size per channel: (13 x 1). Kernel size: (4 x 4). Kernel size can't be greater than actual input size